In [ ]:
#%pip install llama-index-llms-openai
#%pip install llama-index-agent-openai

In [1]:
from dotenv import load_dotenv

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI

/Users/friedatong3906/Desktop/产品分析/corpus/38/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
wiki_titles = ["Boston", "Chicago"]

In [4]:
from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

In [5]:
# Load all wiki documents
city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()

In [6]:
from llama_index.agent.openai import OpenAIAgent

# Build agents dictionary
agents = {}

for wiki_title in wiki_titles:
    # build vector index
    vector_index = VectorStoreIndex.from_documents(
        city_docs[wiki_title],
    )
    # build summary index
    summary_index = SummaryIndex.from_documents(
        city_docs[wiki_title],
    )
    # define query engines
    vector_query_engine = vector_index.as_query_engine()
    list_query_engine = summary_index.as_query_engine()

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    f"Useful for retrieving specific context from {wiki_title}"
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=list_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    "Useful for summarization questions related to"
                    f" {wiki_title}"
                ),
            ),
        ),
    ]

    # build agent
    function_llm = OpenAI(model="gpt-3.5-turbo-0613")
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
    )

    agents[wiki_title] = agent

In [7]:
# define top-level nodes
objects = []
for wiki_title in wiki_titles:
    # define index node that links to these agents
    wiki_summary = (
        f"This content contains Wikipedia articles about {wiki_title}. Use"
        " this index if you need to lookup specific facts about"
        f" {wiki_title}.\nDo not use this index if you want to analyze"
        " multiple cities."
    )
    node = IndexNode(
        text=wiki_summary, index_id=wiki_title, obj=agents[wiki_title]
    )
    objects.append(node)

In [8]:
# define top-level retriever
vector_index = VectorStoreIndex(
    objects=objects,
)
query_engine = vector_index.as_query_engine(similarity_top_k=1, verbose=True)

In [9]:
response = query_engine.query("Tell me which city is more suitable for living, Boston or Chicago")

Retrieval entering Boston: OpenAIAgent
Retrieving from object OpenAIAgent with query Tell me which city is more suitable for living, Boston or Chicago
Added user message to memory: Tell me which city is more suitable for living, Boston or Chicago


In [14]:
query_engine.query("Give me a summary on all the positive aspects of Chicago")

Retrieval entering Chicago: OpenAIAgent
Retrieving from object OpenAIAgent with query Give me a summary on all the positive aspects of Chicago
Added user message to memory: Give me a summary on all the positive aspects of Chicago
=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "positive aspects of Chicago"
}
Got output: Chicago is a bustling city with a dynamic economy that acts as a global center for multiple sectors, including finance, culture, commerce, technology, and transportation. It plays a significant role in the derivatives market and has a substantial gross domestic product. Moreover, the city is a popular tourist spot recognized for its influence on various art forms like visual arts, literature, film, theater, comedy, food, dance, and music. Chicago also houses renowned educational institutions and professional sports teams in major leagues.



Response(response="Chicago is a vibrant and attractive city known for its diverse and robust economy, rich cultural scene, impressive architecture, culinary delights, passionate sports culture, prestigious educational institutions, stunning lakefront views and parks, and efficient public transportation system. These positive aspects collectively contribute to Chicago's dynamic character, making it a desirable destination for residents, tourists, and professionals.", source_nodes=[NodeWithScore(node=TextNode(id_='48b70fc8-9e09-4516-a62e-60c8508ff195', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Chicago has several positive aspects that make it a vibrant and attractive city. Here are some key highlights:\n\n1. Economic Hub: Chicago is a major economic center, with a diverse and robust economy. It is home to numerous Fortune 500 companies and has a strong presence in industries such as finance, technology, manufactur

In [15]:
query_engine.query("Tell me about the sports teams in Boston")

Retrieval entering Boston: OpenAIAgent
Retrieving from object OpenAIAgent with query Tell me about the sports teams in Boston
Added user message to memory: Tell me about the sports teams in Boston


Response(response="Boston is home to several professional sports teams with rich histories and dedicated fan bases. The major sports teams in Boston include the Boston Red Sox in MLB, the New England Patriots in NFL, the Boston Celtics in NBA, the Boston Bruins in NHL, and the New England Revolution in MLS. These teams have achieved significant success in their respective leagues, with multiple championships and a strong presence in the city's sports culture.", source_nodes=[NodeWithScore(node=TextNode(id_='6f488820-10cd-45f6-ade3-5735edcd26a6', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Boston is home to several professional sports teams that have a rich history and a dedicated fan base. Here are some of the major sports teams in Boston:\n\n1. Boston Red Sox (MLB): The Boston Red Sox are one of the oldest and most successful baseball teams in Major League Baseball. They play their home games at Fenway Park, whic

In [16]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='d58e54ad-e840-4a27-902f-8bdbddf47054', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Both Boston and Chicago are vibrant cities with their own unique characteristics. The suitability for living in a city depends on individual preferences and priorities. Here are some factors to consider when comparing Boston and Chicago:\n\n1. Cost of Living: Boston generally has a higher cost of living compared to Chicago. Housing, transportation, and daily expenses tend to be more expensive in Boston.\n\n2. Job Opportunities: Both cities offer a range of job opportunities, but Boston is known for its strong presence in industries like education, healthcare, finance, and technology. Chicago, on the other hand, has a diverse economy with a focus on finance, manufacturing, and transportation.\n\n3. Education: Boston is home to several prestigious universities and colleges, including Harvard University